<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Combining DataFrames
_**Author**: Boom D. (DSI-NYC), Mahdi S. (DSI-NYC)_
***

__First, we'll cover a _simplification_ of the two most common Pandas methods you can combine dataframes together.__

## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # for Pandas plotting

## Loading data

_Note: I've drastically modified and simplified the data from its original source, the [Central Park Squirrel Dataset](https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Squirrel-Data/vfnx-vebw)_

In [2]:
age      = pd.read_csv("./datasets/squirrel_age.csv")
color    = pd.read_csv("./datasets/squirrel_color.csv")
location = pd.read_csv("./datasets/squirrel_location.csv")

In [3]:
age # notice number of observations

,Unique Squirrel ID,Age
0,8A-AM-1013-06,Juvenile
1,7H-PM-1006-07,Adult
2,3G-PM-1013-03,Adult
3,22F-AM-1007-07,Juvenile
4,20A-PM-1017-01,Adult


In [4]:
color # notice number of observations

,Unique Squirrel ID,Primary Fur Color
0,20A-PM-1017-01,Gray
1,22F-AM-1007-07,Cinnamon
2,7H-PM-1006-07,Gray
3,3G-PM-1013-03,Gray


In [5]:
location # notice number of observations

,unique_squirrel_id,lat,long
0,3G-PM-1013-03,-73.974437,40.767428
1,7H-PM-1006-07,-73.970026,40.769934
2,31F-AM-1013-01,-73.959687,40.789379
3,8A-AM-1013-06,-73.977310,40.773805
4,22F-AM-1007-07,-73.964660,40.782770
5,20A-PM-1017-01,-73.970069,40.782889


In [9]:
print(age.shape, color.shape, location.shape)

(5, 2) (4, 2) (6, 3)


Take a moment to notice the shape of these data

## `.merge()`

When we use `.merge()`:
- Only merges 2 dataframes
- We MUST merge on a common column - this is information that is shared by both dataframes.
---
__What is the common column in the `age` and `color` dataframes?__

In [6]:
pd.merge(left = age,
         right = color,
         on = 'Unique Squirrel ID')

,Unique Squirrel ID,Age,Primary Fur Color
0,7H-PM-1006-07,Adult,Gray
1,3G-PM-1013-03,Adult,Gray
2,22F-AM-1007-07,Juvenile,Cinnamon
3,20A-PM-1017-01,Adult,Gray


__Are we missing an observation?__

In [14]:
# Alternative syntax that does the same thing
age.merge(color, on='Unique Squirrel ID')

,Unique Squirrel ID,Age,Primary Fur Color
0,7H-PM-1006-07,Adult,Gray
1,3G-PM-1013-03,Adult,Gray
2,22F-AM-1007-07,Juvenile,Cinnamon
3,20A-PM-1017-01,Adult,Gray


### What if we reverse the input order? What changes?

In [7]:
pd.merge(left = color,
         right = age,
         on = "Unique Squirrel ID")

,Unique Squirrel ID,Primary Fur Color,Age
0,20A-PM-1017-01,Gray,Adult
1,22F-AM-1007-07,Cinnamon,Juvenile
2,7H-PM-1006-07,Gray,Adult
3,3G-PM-1013-03,Gray,Adult


### What if I don't want the _intersection_ and, instead, I want to keep everything from the right table (i.e. `age`, the bigger one)?

In [8]:
pd.merge(left = color,
         right = age,
         how = "right",
         on = "Unique Squirrel ID")

,Unique Squirrel ID,Primary Fur Color,Age
0,8A-AM-1013-06,NaN,Juvenile
1,7H-PM-1006-07,Gray,Adult
2,3G-PM-1013-03,Gray,Adult
3,22F-AM-1007-07,Cinnamon,Juvenile
4,20A-PM-1017-01,Gray,Adult


Using `how="right"`, what's changed?

### What if I have a dataframe with a _different_ name for the column I wish to join "on"?

In [10]:
age

,Unique Squirrel ID,Age
0,8A-AM-1013-06,Juvenile
1,7H-PM-1006-07,Adult
2,3G-PM-1013-03,Adult
3,22F-AM-1007-07,Juvenile
4,20A-PM-1017-01,Adult


In [11]:
location

,unique_squirrel_id,lat,long
0,3G-PM-1013-03,-73.974437,40.767428
1,7H-PM-1006-07,-73.970026,40.769934
2,31F-AM-1013-01,-73.959687,40.789379
3,8A-AM-1013-06,-73.977310,40.773805
4,22F-AM-1007-07,-73.964660,40.782770
5,20A-PM-1017-01,-73.970069,40.782889


In [12]:
# This breaks...
pd.merge(left = age,
         right = location,
         on = "Unique Squirrel ID")

KeyError: 'Unique Squirrel ID'

In [13]:
# This WORKS!
pd.merge(left = age,
         right = location,
         left_on = 'Unique Squirrel ID',
         right_on = 'unique_squirrel_id')

,Unique Squirrel ID,Age,unique_squirrel_id,lat,long
0,8A-AM-1013-06,Juvenile,8A-AM-1013-06,-73.977310,40.773805
1,7H-PM-1006-07,Adult,7H-PM-1006-07,-73.970026,40.769934
2,3G-PM-1013-03,Adult,3G-PM-1013-03,-73.974437,40.767428
3,22F-AM-1007-07,Juvenile,22F-AM-1007-07,-73.964660,40.782770
4,20A-PM-1017-01,Adult,20A-PM-1017-01,-73.970069,40.782889


We see some redundancy, which is working as expected...
- You may have code that breaks if it expects some incoming datafame to have the specific column "unique_squirrel_id" in some place and "Unique Squirrel ID" in others

## `.concat()`

#### Concatenating by columns _(not recommended)_

In [ ]:
# axis 1 => by column, axis 0 => by row


Notice how we can concatenate two dataframes without the same number of rows, but...
- The overlap is filled with `NaN` values

### Can we `.concat()` more than 2 dataframes?

#### Concatenating by rows _(useful)_

In [15]:
# Creating a new data point (row)
new_datapoint = pd.DataFrame(data = [['8A-AM-1013-06', "Cinnamon"]],
                             columns = ['Unique Squirrel ID', 'Primary Fur Color'])


In [16]:
new_datapoint

,Unique Squirrel ID,Primary Fur Color
0,8A-AM-1013-06,Cinnamon


In [17]:
# Concatenate new datapoint to existing dataframe
new_color = pd.concat(objs = [color, new_datapoint], axis = 0)
new_color

,Unique Squirrel ID,Primary Fur Color
0,20A-PM-1017-01,Gray
1,22F-AM-1007-07,Cinnamon
2,7H-PM-1006-07,Gray
3,3G-PM-1013-03,Gray
0,8A-AM-1013-06,Cinnamon


__Is there anything odd about this new dataframe?__

In [ ]:
# Reset index


In [ ]:
new_color

## References
- [Central Park Squirrel Census](https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Squirrel-Data/vfnx-vebw)